In [1]:
!pip install transformers datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import wandb

# Loading mini Flan dataset

In [3]:
df = pd.read_parquet("train-00000-of-00001.parquet")

In [4]:
df = df.iloc[:, :2]
df

,inputs,targets
0,"The thyroid gland, or simply the thyroid, is a...",no
1,Hayes High School (Birmingham Alabama) - Carol...,Animal
2,Answer the following question: Read the articl...,D
3,Please answer this: what is the charge for ent...,"June 24, 2017"
4,Write down the solution for this math problem:...,-5
...,...,...
143995,Problem: Build a movie plot around this: What ...,"In an unidentified city a gangster, Eddie Blac..."
143996,input question: Generate a question that has t...,What does Fiona do right before Shrek disappears?
143997,input: Please answer the following: Question: ...,Castle of Chaythe
143998,Build a movie plot around this: What has happe...,Despairing over the loss of a 19-year-old Mari...


# prompt

In [5]:
print(df.iloc[450,0])

Archaeological evidence shows that Homo erectus lived in the region now known as Myanmar as early as 400,000 years ago. The first evidence of Homo sapiens is dated to about 11,000 BC, in a Stone Age culture called the Anyathian with discoveries of stone tools in central Myanmar. Evidence of neolithic age domestication of plants and animals and the use of polished stone tools dating to sometime between 10,000 and 6,000 BC has been discovered in the form of cave paintings near the city of Taunggyi.

Answer this question, if possible (if impossible, reply "unanswerable"): What form was the evidence of ancient cultures discovered in ?


# target

In [6]:
print(df.iloc[450,1])

discovered in the form of cave paintings


source:`https://huggingface.co/datasets/pszemraj/flan-subsets-mini`

some ref:`https://huggingface.co/BEE-spoke-data/tFINE-680m-e32-d16-gqa-flan`

#for the forward

In [8]:
dataset = Dataset.from_pandas(df)

In [7]:
model_name = "EleutherAI/pythia-160m"  #select the lm model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = "<pad>"  # seems without custom pad token set to zero the convergence behaves weirdly
tokenizer.pad_token_id = 0     # setting pad_token_id to 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

`note that in the Mini Flan dataset, the instruction and question are already merged into one as mentioned in the paper pg 5 for fine tuning`

`Original Sequence: What is the capital of France? → Paris`

`Tokenized input_ids: [A', B', C', D', E', F', G', H']`

`Correct labels: [B', C', D', E', F', G', H', -100'] (shift left).`

In [26]:
#forward tokenization
def preprocess_forward(examples):
    combined_text = [f"{inp}\n{target}" for inp, target in zip(examples['inputs'], examples['targets'])]
    tokenized_output = tokenizer(combined_text, truncation=True, padding="max_length", max_length=768, return_tensors="pt")
    input_ids = tokenized_output["input_ids"]
    labels = input_ids.clone()
    labels[:, :-1] = input_ids[:, 1:]  # shifting tokens to the right
    labels[:, -1] = -100  # the last token (no next token to predict) ignored
    tokenized_output["labels"] = labels
    return tokenized_output

tokenized_forward = dataset.map(preprocess_forward, batched=True)

Map:   0%|          | 0/144000 [00:00<?, ? examples/s]

In [27]:
#shuffle the data
tokenized_forward = tokenized_forward.shuffle(seed=42)

In [28]:
print("\nForward Direction - Tokenized Examples:")
for i in range(3):
    print(f"Example {i + 1}:")
    print(f"Input IDs: {tokenized_forward[i]['input_ids']}")
    print(f"Decoded Tokens: {tokenizer.decode(tokenized_forward[i]['input_ids'], skip_special_tokens=True)}")
    print("-" * 50)


Forward Direction - Tokenized Examples:
Example 1:
Input IDs: [36798, 27, 19566, 247, 2762, 340, 47705, 2278, 15, 187, 34, 27, 831, 310, 247, 1175, 309, 41, 2795, 4328, 15, 380, 4750, 369, 1077, 11453, 285, 9371, 15, 380, 17552, 2170, 310, 5322, 285, 4076, 13, 285, 253, 7180, 497, 4076, 285, 9848, 15, 380, 2579, 369, 3468, 13, 533, 5010, 597, 497, 1077, 1175, 15, 187, 187, 36798, 27, 19566, 247, 4016, 340, 47705, 2278, 15, 187, 34, 27, 8948, 352, 434, 619, 2363, 390, 253, 958, 326, 776, 1387, 369, 22813, 1735, 281, 247, 1387, 273, 952, 326, 3261, 751, 597, 12293, 562, 273, 253, 5579, 921, 3052, 763, 353, 41067, 326, 574, 20466, 594, 3076, 326, 359, 2546, 281, 320, 4395, 13, 533, 436, 921, 369, 11527, 2, 329, 2069, 309, 1119, 253, 16226, 594, 19328, 309, 3078, 281, 3835, 619, 2454, 15, 380, 2398, 1031, 79, 34334, 1607, 3796, 1080, 671, 25128, 347, 1608, 16642, 4867, 404, 13, 534, 309, 1119, 37750, 846, 309, 4720, 8156, 313, 76, 438, 375, 281, 617, 323, 10480, 272, 441, 481, 380, 45655,

In [29]:
#split the dataset
tokenized_forward = tokenized_forward.train_test_split(test_size=0.1)  # 90% train, 10% test

In [30]:
# loading the model for fine tuning
model_forward = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")

# hyperparameter config
training_args = TrainingArguments(
    output_dir="./fine-tuned-pythia-forward",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.05,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs-forward",
    logging_steps=10,
    push_to_hub=False,
)

# initilize of train
trainer_forward = Trainer(
    model=model_forward,
    args=training_args,
    train_dataset=tokenized_forward["train"],
    eval_dataset=tokenized_forward["test"],
    tokenizer=tokenizer,
)

trainer_forward.train()

# saving the model
model_forward.save_pretrained("./fine-tuned-pythia-forward")
tokenizer.save_pretrained("./fine-tuned-pythia-forward")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-9ae4863075dc>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_forward = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: janbol (11785_finetuning) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [31]:
wandb.finish()

train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇██
train/grad_norm,█▃▄▅▃▃▅▂▄▅▃▄▄▄▄▄▄▃▃▃▃▃▃▄▃▃▂▃▂▄▂▃▃▂▃▃▃▂▁▂
train/learning_rate,███▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▃▂▂▄▃▂▁▃▂▂▂▂▁▁▂▂▁▂▁▁▂▂▂▁▂▁▁▂▂▁▁▂▁▁▁▂▁▂▂
train/epoch,0.04568
train/global_step,740
train/grad_norm,6.06754
train/learning_rate,1e-05
train/loss,2.7152


# for the backward

`Original Sequence: What is the capital of France? → Paris`

`Reversed Sequence: France of capital the is What → Paris`

`Tokenized input_ids: [A', B', C', D', E', F', G', H']`

`[H', G', F', E', D', C', B', A']`

`Correct labels: [-100', A', B', C', D', E', F', G'] (shift right).`

`[G', F', E', D', C', B', A', -100']`

In [32]:
model_name = "afterless/reverse-pythia-160m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = "<pad>"  # seems without custom pad token set to zero the convergence behaves weirdly
tokenizer.pad_token_id = 0     # setting pad_token_id to 0

config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [33]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer for this model doesn't have padding token, add explicit custom padding
def preprocess_reverse(examples):
    # revering the order of inputs and targets at WORD level not char level
    reversed_inputs = [" ".join(inp.split()[::-1]) for inp in examples['inputs']]
    reversed_targets = [" ".join(target.split()[::-1]) for target in examples['targets']]
    combined_text = [f"{inp}\n{target}" for inp, target in zip(reversed_targets,reversed_inputs)]
    # tokenize
    tokenized_output = tokenizer(combined_text, truncation=True, padding="max_length", max_length=768, return_tensors="pt", padding_side="left")

    # shifting input_ids to create labels (shift left)
    input_ids = tokenized_output["input_ids"]
    labels = input_ids.clone() #copy
    labels[:, 1:] = input_ids[:, 1:]  # shift tokens to the right
    labels[:, -1] = -100
    tokenized_output["labels"] = labels

    return tokenized_output

In [34]:
tokenized_backward = dataset.map(preprocess_reverse, batched=True)

Map:   0%|          | 0/144000 [00:00<?, ? examples/s]

In [35]:
#shuffle the data
tokenized_backward = tokenized_backward.shuffle(seed=42)

In [37]:
print("Backward Direction - Tokenized Examples:")
for i in range(3):
    print(f"Example {i + 1}:")
    print(f"Input IDs: {tokenized_backward[i]['input_ids']}")
    print(f"Decoded Tokens: {tokenizer.decode(tokenized_backward[i]['input_ids'], skip_special_tokens=True)}")
    print("-" * 50)

Backward Direction - Tokenized Examples:
Example 1:
Input IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [38]:
tokenized_backward = tokenized_backward.train_test_split(test_size=0.1)

In [39]:
# loading the model for fine tuning
model_backward = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./fine-tuned-pythia-backward",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs-backward",
    logging_steps=10,
    push_to_hub=False
)

# initilize of train
trainer_backward = Trainer(
    model=model_backward,
    args=training_args,
    train_dataset=tokenized_backward["train"],
    eval_dataset=tokenized_backward["test"],
    tokenizer=tokenizer,
)

#.forward
trainer_backward.train()

# saving the model
model_backward.save_pretrained("./fine-tuned-pythia-backward")
tokenizer.save_pretrained("./fine-tuned-pythia-backward")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-39-d100b1a73237>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_backward = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [40]:
wandb.finish()

train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇████
train/grad_norm,█▁▆▆▆▂▄█▅▆█▄▅▅▆▄▆▇▆▄▅▆▅▅▅▂▄▅▄▄▆▂▄▂▃▃▆▆▁▄
train/learning_rate,███▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁
train/loss,█▂▂▂▂▁▂▂▂▂▁▁▂▁▁▁▁▂▂▁▁▁▁▂▁▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁
train/epoch,0.04568
train/global_step,740
train/grad_norm,3.24055
train/learning_rate,1e-05
train/loss,1.5851
